# Milk-in-Coffee Loschmidt Echo (X-θ Test)

## Goal

Test whether an extra compact fiber degree of freedom (θ) can act as a **hidden information channel** that makes macroscopic dynamics appear irreversible when θ is uncontrolled — while remaining reversible when θ is controlled.

We use:

- **Coffee stirring**: real velocity fields u(x,y,t), v(x,y,t) from The Well
- **Milk**: passive scalar concentration C(x,y,t)
- **X-θ coupling**: extra drift velocity  
  v_Xθ(x,y,t) = κ ∇Aθ(x,y) · θ̇(t)

## Key idea (Loschmidt Echo)

1. Forward evolve for T/2 steps.
2. Reverse the _observable_ flow field (u,v) and evolve backward for T/2 steps.
3. Measure how well we recover the initial milk blob.

## Reversal Modes (Critical Control)

- **perfect**: reverse base flow AND reverse θ protocol (θ̇ → −θ̇).  
  Expect: high echo fidelity (limited only by numerics/mixing).
- **hidden**: reverse base flow only, θ is not reversed / uncontrolled.  
  Expect: echo degrades in a systematic way that scales with winding N.

## Primary prediction (Falsifiable)

Echo degradation depends on **winding number N** and flips sign under N → −N (where applicable).
If not, X-θ mechanism is not supported by this model.


In [4]:
!pip -q install -U pip
!pip -q install the-well torch numpy matplotlib
# Optional (only needed if you later switch to local HDF5 validation/repair)
!pip -q install -U huggingface_hub fsspec aiohttp hf_transfer
!pip -q install h5py




ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
the-well 1.2.0 requires fsspec==2024.10.0, but you have fsspec 2025.12.0 which is incompatible.


In [17]:
import sys, subprocess

def pip_install(*args):
    subprocess.check_call([sys.executable, "-m", "pip", "install", *args])

pip_install("-U", "pip", "setuptools", "wheel")
pip_install("-U", "huggingface_hub", "fsspec", "aiohttp")

# Optional but helpful for faster/more robust downloads/streaming
pip_install("-U", "hf_transfer")

print("Done.")


Done.


In [18]:
import sys, platform
import torch, numpy as np
import matplotlib
import huggingface_hub, fsspec

print("Python:", sys.version)
print("Platform:", platform.platform())
print("torch:", torch.__version__, "| cuda:", torch.cuda.is_available())
print("numpy:", np.__version__)
print("matplotlib:", matplotlib.__version__)
print("huggingface_hub:", huggingface_hub.__version__)
print("fsspec:", fsspec.__version__)


Python: 3.13.5 (tags/v3.13.5:6cb20a2, Jun 11 2025, 16:15:46) [MSC v.1943 64 bit (AMD64)]
Platform: Windows-11-10.0.26200-SP0
torch: 2.9.1+cpu | cuda: False
numpy: 2.3.5
matplotlib: 3.10.8
huggingface_hub: 1.2.3
fsspec: 2025.12.0


In [19]:
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"
print("HF_HUB_ENABLE_HF_TRANSFER =", os.environ["HF_HUB_ENABLE_HF_TRANSFER"])




HF_HUB_ENABLE_HF_TRANSFER = 1


In [20]:
import platform
IS_WINDOWS = (platform.system().lower() == "windows")
print("Platform:", platform.system(), "| IS_WINDOWS:", IS_WINDOWS)

# Workaround: ensure The Well doesn't build hf paths using backslashes on Windows.
# This patches only the_well module's join usage (does not globally change os.path).
if IS_WINDOWS:
    import posixpath
    import the_well.data.datasets as dsmod
    dsmod.os.path.join = posixpath.join
    print("Patched the_well.data.datasets.os.path.join -> posixpath.join")


Platform: Windows | IS_WINDOWS: True
Patched the_well.data.datasets.os.path.join -> posixpath.join


In [21]:
import math
import numpy as np
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

from the_well.data import WellDataset

device = "cuda" if torch.cuda.is_available() else "cpu"
print("torch:", torch.__version__, "| device:", device)


torch: 2.9.1+cpu | device: cpu


In [22]:
from the_well.data import WellDataset

DATASET = "turbulent_radiative_layer_2D"
SPLIT   = "train"
T_IN    = 32
T_OUT   = 32
WINDOW_INDEX = 0

ds = WellDataset(
    well_base_path="hf://datasets/polymathic-ai/",
    well_dataset_name=DATASET,
    well_split_name=SPLIT,
    n_steps_input=T_IN,
    n_steps_output=T_OUT,
    use_normalization=False,
)

item = ds[WINDOW_INDEX]
print("✅ Loaded:", DATASET, SPLIT)
print("input_fields:", item["input_fields"].shape)
print("output_fields:", item["output_fields"].shape)


✅ Loaded: turbulent_radiative_layer_2D train
input_fields: torch.Size([32, 128, 384, 4])
output_fields: torch.Size([32, 128, 384, 4])


In [23]:
print("metadata keys groups:", list(ds.metadata.field_names.keys()))
print("field names (first 30):", [n for g in ds.metadata.field_names.values() for n in g][:30])


metadata keys groups: [0, 1, 2]
field names (first 30): ['density', 'pressure', 'velocity_x', 'velocity_y']


In [24]:
def flatten_field_names(field_names_dict):
    out = []
    for _, group in field_names_dict.items():
        out.extend(group)
    return out

field_names = flatten_field_names(ds.metadata.field_names)
print("Flattened field names:", field_names)

def find_velocity_indices(names):
    pairs = [
        ("velocity_x", "velocity_y"),
        ("u", "v"),
        ("vel_x", "vel_y"),
        ("vx", "vy"),
    ]
    for a, b in pairs:
        if a in names and b in names:
            return names.index(a), names.index(b)

    vel = [i for i, n in enumerate(names) if "velocity" in n.lower()]
    if len(vel) >= 2:
        return vel[0], vel[1]

    raise RuntimeError("No velocity channels found.\n" + "\n".join(names))

ix, iy = find_velocity_indices(field_names)
print("Using velocity channels:", field_names[ix], field_names[iy])


Flattened field names: ['density', 'pressure', 'velocity_x', 'velocity_y']
Using velocity channels: velocity_x velocity_y


1. A_MODE = "ramp" (calibration mode)

𝐴
𝜃
(
𝑥
,
𝑦
)
=
𝛼

𝑥
A
θ
​

(x,y)=αx

∇
𝐴
𝜃
∇A
θ
​

is basically a constant shove to the right.

Good for: “does the wiring work?” and clean
Δ
∝
𝑁
Δ∝N scaling.

2. A_MODE = "sine" (geometric / structured mode)

𝐴
𝜃
(
𝑥
,
𝑦
)
=
𝛼
sin
⁡
(
2
𝜋
(
𝑘
𝑥
𝑥

- 𝑘
  𝑦
  𝑦
  )
  )
  A
  θ
  ​

(x,y)=αsin(2π(k
x
​

x+k
y
​

y))

∇
𝐴
𝜃
∇A
θ
​

varies across space (pushes different directions in different regions).

Good for: showing this isn’t just a constant drift; it produces spatially patterned effects.

3. A_MODE = "field" (data-driven mode, best if you want realism)

Use an actual scalar field from the dataset (e.g., temperature, density, pressure) as
𝐴
𝜃
A
θ
​

:

normalize it, then
𝐴
𝜃
=
𝛼
⋅
normalized(field)
A
θ
​

=α⋅normalized(field)

Good for: “X-θ couples to the medium,” and it ties the effect to real physics content.

What you should use, in practice

Start with "ramp" to validate the echo + winding dependence.

Then go to "sine" to show it survives when
𝐴
𝜃
A
θ
​

is not trivial.

Then, if the dataset has a suitable scalar channel, use "field" for the most convincing story.

Tiny cheat sheet of knobs

ALPHA sets the strength of
𝐴
𝜃
A
θ
​

(and its gradient).

A_KX, A_KY (only for "sine") set how many waves fit across the domain.

KAPPA scales the coupling strength.

If you paste the printed Flattened field names: list from your run, I’ll tell you exactly which channel is most sensible to use for A_MODE="field" in that dataset.


In [2]:
CONFIG = dict(
    DATASET="turbulent_radiative_layer_2D",
    SPLIT="train",
    T_IN=32,
    T_OUT=32,

    # windows = different rollouts/samples, used for error bars
    WINDOW_INDICES=list(range(0, 8)),   # 0..7

    # X-θ knobs
    KAPPA=30.0,         # coupling strength
    ALPHA=10.0,         # Aθ strength
    A_MODE="field",     # "field" (best), "sine", "ramp"
    A_KX=4,
    A_KY=0,
    A_FIELD_NAME=None,  # if None and A_MODE="field", we auto-pick a scalar channel

    # advection / scaling knobs
    VEL_SCALE=1.0,      # scale velocity if echo too weak/strong
    NOISE_VEL=0.0,      # imperfect reversal noise (0 to start)
    MILK_SIGMA=10.0,    # blob size

    # winding list
    N_LIST=[-8,-4,-2,-1,0,1,2,4,8],
)
CONFIG


{'DATASET': 'turbulent_radiative_layer_2D',
 'SPLIT': 'train',
 'T_IN': 32,
 'T_OUT': 32,
 'WINDOW_INDICES': [0, 1, 2, 3, 4, 5, 6, 7],
 'KAPPA': 30.0,
 'ALPHA': 10.0,
 'A_MODE': 'field',
 'A_KX': 4,
 'A_KY': 0,
 'A_FIELD_NAME': None,
 'VEL_SCALE': 1.0,
 'NOISE_VEL': 0.0,
 'MILK_SIGMA': 10.0,
 'N_LIST': [-8, -4, -2, -1, 0, 1, 2, 4, 8]}

In [3]:
ds = WellDataset(
    well_base_path="hf://datasets/polymathic-ai/",
    well_dataset_name=CONFIG["DATASET"],
    well_split_name=CONFIG["SPLIT"],
    n_steps_input=CONFIG["T_IN"],
    n_steps_output=CONFIG["T_OUT"],
    use_normalization=False,
)

print("Loaded dataset:", CONFIG["DATASET"], CONFIG["SPLIT"])
print("Example item shapes:",
      ds[0]["input_fields"].shape,
      ds[0]["output_fields"].shape)


NameError: name 'WellDataset' is not defined

In [26]:
def metrics(C_final, C0):
    # correlation fidelity
    F = fidelity_corr(C_final, C0)

    # L2 relative error
    diff = (C_final - C0)
    l2 = torch.linalg.norm(diff.flatten())
    l2_rel = float((l2 / torch.linalg.norm(C0.flatten()).clamp_min(1e-12)).detach().cpu().item())

    # "mass" conservation sanity (should be roughly stable under semi-Lagrangian)
    m0 = float(C0.sum().detach().cpu().item())
    m1 = float(C_final.sum().detach().cpu().item())
    mass_err = (m1 - m0) / (abs(m0) + 1e-12)

    return dict(F=F, l2_rel=l2_rel, mass_err=mass_err)


In [27]:
import torch
import numpy as np

def get_grid(H, W, device):
    y = torch.linspace(0, 1, H, device=device)
    x = torch.linspace(0, 1, W, device=device)
    yy, xx = torch.meshgrid(y, x, indexing="ij")
    return yy, xx

def make_milk_blob(H, W, device, sigma=10.0):
    """Creates a Gaussian blob in the center of the grid."""
    yy, xx = get_grid(H, W, device)
    cy, cx = 0.5, 0.5
    # sigma is in pixels, so convert to 0..1 scale relative to height
    s = sigma / H 
    blob = torch.exp(-((xx - cx)**2 + (yy - cy)**2) / (2 * s**2))
    return blob

def build_A_theta(mode, H, W, device, alpha, kx=4, ky=0):
    """Constructs the gauge potential A_theta(x, y)."""
    yy, xx = get_grid(H, W, device)
    
    if mode == "ramp":
        # Gradient is constant (1, 0)
        return alpha * xx
    elif mode == "sine":
        # Sinusoidal potential
        return alpha * torch.sin(2 * np.pi * (kx * xx + ky * yy))
    elif mode == "field":
        # Placeholder for data-driven field; defaults to ramp if not fed explicit field
        # In a full implementation, you'd pass a scalar field here.
        return alpha * xx 
    else:
        raise ValueError(f"Unknown A_mode: {mode}")

def semi_lagrangian_step(C, vx, vy, dt):
    """Advects scalar C by velocity (vx, vy) using semi-Lagrangian interpolation."""
    H, W = C.shape
    device = C.device
    
    # Create normalized grid [-1, 1] for grid_sample
    yy, xx = get_grid(H, W, device) # 0..1
    
    # Back-trace: where did the particle come from?
    # Velocity is in units of [0..1] per time unit if vel_scale=1.0 and grid is 0..1
    # We generally assume vx, vy are normalized or in pixels. 
    # Let's assume input vx, vy are in grid-units/sec, so we scale to normalized coords.
    
    # Note: grid_sample expects (x, y) last dimension, in range [-1, 1]
    # Current grid is 0..1. Map to -1..1
    grid_x = (xx * 2 - 1)
    grid_y = (yy * 2 - 1)
    
    # Displace
    # We need velocity in "normalized grid units". 
    # If vx is pixels/step, normalized vel is vx * (2/W).
    # Here we assume vx, vy are provided in a compatible scale.
    # For safety in these demos, we often assume vx, vy are raw magnitudes roughly ~1.0 max.
    
    prev_x = grid_x - vx * dt * 2 # factor 2 because range is 2 (-1 to 1)
    prev_y = grid_y - vy * dt * 2
    
    # Stack for grid_sample: (1, H, W, 2)
    grid = torch.stack((prev_x, prev_y), dim=-1).unsqueeze(0)
    
    # Sample
    C_in = C.unsqueeze(0).unsqueeze(0) # (1, 1, H, W)
    C_new = torch.nn.functional.grid_sample(C_in, grid, align_corners=True, padding_mode="border")
    
    return C_new.squeeze()

def fidelity_corr(C1, C2):
    """Calculates correlation fidelity between two fields."""
    c1 = C1.flatten()
    c2 = C2.flatten()
    num = torch.dot(c1, c2)
    denom = torch.linalg.norm(c1) * torch.linalg.norm(c2) + 1e-12
    return (num / denom).item()

def run_echo(vel_seq, A_theta, N_wind, kappa, dt, reverse_mode, vel_scale, C0, noise_vel=0.0):
    """
    Runs the forward and backward evolution (Loschmidt Echo).
    
    Args:
        vel_seq: (T, H, W, 2) tensor of background velocity.
        A_theta: (H, W) tensor of gauge potential.
        N_wind: Total winding number (turns of theta).
        kappa: Coupling strength.
        dt: Time step.
        reverse_mode: "perfect" (reverse theta) or "hidden" (don't reverse theta).
        vel_scale: Multiplier for background velocity.
        C0: Initial milk blob.
        noise_vel: Magnitude of noise to add.
        
    Returns:
        F_control, F_xtheta, C_control_final, C_xtheta_final
    """
    T, H, W, _ = vel_seq.shape
    half_T = T // 2
    
    # Pre-calculate gradients of A_theta for the X-theta drift
    # grad_A[0] is dy, grad_A[1] is dx in torch.gradient
    grad_y, grad_x = torch.gradient(A_theta) 
    
    # Prepare standard (control) and X-theta (experiment) blobs
    C_ctrl = C0.clone()
    C_exp  = C0.clone()
    
    # Define phase velocity theta_dot
    # We want integral(theta_dot dt) = N_wind * 2pi over full time T? 
    # Usually in echo: Forward for T/2, Backward for T/2.
    # Let's say we wind N times over the full experiment, or N during forward?
    # Standard echo: Winding N forward, then unwind -N backward (perfect) or continues +N (hidden).
    # Rate: omega = (N * 2pi) / (T * dt) roughly.
    omega = (N_wind * 2 * np.pi) / (T * dt)
    
    # --- FORWARD PASS (0 to T/2) ---
    for t in range(half_T):
        vx = vel_seq[t, ..., 0] * vel_scale
        vy = vel_seq[t, ..., 1] * vel_scale
        
        # X-theta drift: v_drift = kappa * grad A * theta_dot
        # For forward pass, theta_dot = omega
        drift_x = kappa * grad_x * omega
        drift_y = kappa * grad_y * omega
        
        # Evolve Control (Base flow only)
        C_ctrl = semi_lagrangian_step(C_ctrl, vx, vy, dt)
        
        # Evolve Experiment (Base + Drift)
        C_exp = semi_lagrangian_step(C_exp, vx + drift_x, vy + drift_y, dt)

    # --- BACKWARD PASS (T/2 to T) ---
    # We reverse the base flow sequence: T/2-1 down to 0
    # And flip velocity sign
    
    for t in range(half_T - 1, -1, -1):
        vx = -vel_seq[t, ..., 0] * vel_scale
        vy = -vel_seq[t, ..., 1] * vel_scale
        
        # Determine theta_dot for backward pass
        if reverse_mode == "perfect":
            # Time reversal succeeds: we reverse the drive
            theta_dot_bwd = -omega
        elif reverse_mode == "hidden":
            # Time reversal fails: drive continues forward (or is uncontrolled)
            # Typically "hidden" means we cannot reverse the global drive phase velocity.
            theta_dot_bwd = omega 
        else:
            theta_dot_bwd = 0.0

        drift_x = kappa * grad_x * theta_dot_bwd
        drift_y = kappa * grad_y * theta_dot_bwd
        
        # Evolve
        C_ctrl = semi_lagrangian_step(C_ctrl, vx, vy, dt)
        C_exp = semi_lagrangian_step(C_exp, vx + drift_x, vy + drift_y, dt)
        
    return fidelity_corr(C_ctrl, C0), fidelity_corr(C_exp, C0), C_ctrl, C_exp

In [28]:
def run_sweep_across_windows(
    window_indices,
    reverse_mode,
    N_list,
    kappa,
    alpha,
    A_mode="ramp",
    vel_scale=1.0,
    noise_vel=0.0,
    milk_sigma=10.0,
):
    all_rows = []

    for widx in window_indices:
        it = ds[widx]
        fields_w = torch.cat([it["input_fields"], it["output_fields"]], dim=0).to(torch.float32)
        vel_w = fields_w[..., [ix, iy]].to(device)

        # dt (use same logic)
        dt_w = 1.0
        t_in = it.get("input_time_grid", None)
        t_out = it.get("output_time_grid", None)
        if t_in is not None and t_out is not None:
            t_full = torch.cat([t_in, t_out], dim=0).to(torch.float32)
            if len(t_full) >= 2:
                dt_w = float((t_full[1] - t_full[0]).abs().cpu().item()) or 1.0

        T, H, W, _ = vel_w.shape
        C0_w = make_milk_blob(H, W, device=device, sigma=milk_sigma)

        A_theta_w = build_A_theta(A_mode, H, W, device=device, alpha=alpha, kx=4, ky=0)

        for N in N_list:
            fc, fx, Cc, Cx = run_echo(
                vel_seq=vel_w,
                A_theta=A_theta_w,
                N_wind=N,
                kappa=kappa,
                dt=dt_w,
                reverse_mode=reverse_mode,
                vel_scale=vel_scale,
                C0=C0_w,
                noise_vel=noise_vel,
            )
            mc = metrics(Cc, C0_w)
            mx = metrics(Cx, C0_w)

            all_rows.append(dict(
                window=widx, reverse_mode=reverse_mode, N=N,
                F_control=mc["F"], F_xtheta=mx["F"],
                l2_control=mc["l2_rel"], l2_xtheta=mx["l2_rel"],
                masserr_control=mc["mass_err"], masserr_xtheta=mx["mass_err"],
            ))

    return all_rows

# Try 5–10 windows to start (increase later)
WINDOWS = list(range(0, 6))  # 0..5
rows_hidden  = run_sweep_across_windows(WINDOWS, "hidden",  N_LIST, KAPPA, ALPHA, A_MODE, VEL_SCALE, NOISE_VEL, 10.0)
rows_perfect = run_sweep_across_windows(WINDOWS, "perfect", N_LIST, KAPPA, ALPHA, A_MODE, VEL_SCALE, NOISE_VEL, 10.0)

len(rows_hidden), len(rows_perfect)


'[WinError 10054] An existing connection was forcibly closed by the remote host' thrown while requesting GET https://huggingface.co/datasets/polymathic-ai/turbulent_radiative_layer_2D/resolve/main/data/train/turbulent_radiative_layer_tcool_0.03.hdf5
Retrying in 1s [Retry 1/5].
'[WinError 10054] An existing connection was forcibly closed by the remote host' thrown while requesting GET https://huggingface.co/datasets/polymathic-ai/turbulent_radiative_layer_2D/resolve/main/data/train/turbulent_radiative_layer_tcool_0.03.hdf5
Retrying in 2s [Retry 2/5].


(30, 30)

In [29]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(rows_hidden + rows_perfect)

agg = (
    df.groupby(["reverse_mode", "N"])
      .agg(F_mean=("F_xtheta", "mean"),
           F_std=("F_xtheta", "std"),
           l2_mean=("l2_xtheta", "mean"),
           l2_std=("l2_xtheta", "std"),
           n=("F_xtheta", "count"))
      .reset_index()
)
agg["F_sem"]  = agg["F_std"] / np.sqrt(agg["n"].clip(lower=1))
agg["l2_sem"] = agg["l2_std"] / np.sqrt(agg["n"].clip(lower=1))
agg


ModuleNotFoundError: No module named 'pandas'

In [ ]:
def plot_metric(metric_mean, metric_sem, title, ylabel):
    plt.figure()
    for mode in ["hidden", "perfect"]:
        sub = agg[agg["reverse_mode"] == mode].sort_values("N")
        plt.errorbar(sub["N"], sub[metric_mean], yerr=sub[metric_sem], marker="o", capsize=3, label=mode)
    plt.xlabel("Winding number N")
    plt.ylabel(ylabel)
    plt.title(title)
    plt.grid(True)
    plt.legend()
    plt.show()

plot_metric("F_mean", "F_sem", "Loschmidt Echo Fidelity vs N (mean±SEM)", "Fidelity (corr)")
plot_metric("l2_mean", "l2_sem", "Relative L2 Error vs N (mean±SEM)", "Relative L2 error")


In [30]:
def fit_line(Ns, Ys):
    Ns = np.array(Ns, dtype=float)
    Ys = np.array(Ys, dtype=float)
    A = np.vstack([Ns, np.ones_like(Ns)]).T
    m, b = np.linalg.lstsq(A, Ys, rcond=None)[0]
    pred = m*Ns + b
    ss_res = np.sum((Ys - pred)**2)
    ss_tot = np.sum((Ys - Ys.mean())**2) + 1e-12
    r2 = 1 - ss_res/ss_tot
    return m, b, r2, pred

# Use mean fidelity difference between perfect and hidden as the "signal"
pivot = agg.pivot(index="N", columns="reverse_mode", values="F_mean").reset_index()
pivot["signal"] = pivot["perfect"] - pivot["hidden"]
pivot = pivot.sort_values("N")

m, b, r2, pred = fit_line(pivot["N"], pivot["signal"])
print("Signal fit: signal ≈ m*N + b")
print("m =", m, "b =", b, "R^2 =", r2)

plt.figure()
plt.plot(pivot["N"], pivot["signal"], marker="o", label="signal (perfect - hidden)")
plt.plot(pivot["N"], pred, label="linear fit")
plt.xlabel("N")
plt.ylabel("Echo signal")
plt.title("Topological lever: linear-in-N test")
plt.grid(True)
plt.legend()
plt.show()

# Odd symmetry test (N vs -N)
sym = pivot.set_index("N")["signal"].to_dict()
pairs = [(n, -n) for n in pivot["N"] if -n in sym and n > 0]
odd_errors = [abs(sym[n] + sym[-n]) for n, _ in pairs]
print("Odd-symmetry |signal(N)+signal(-N)| (mean):", float(np.mean(odd_errors)) if odd_errors else None)


NameError: name 'agg' is not defined

# Proposed Experiment (Simulation → Lab)

## Simulation Observable

Loschmidt echo fidelity:
F = ⟨C_final, C0⟩ / (||C_final|| ||C0||)

We compare:

- Control (κ=0)
- X-θ perfect reversal (reverse flow and θ protocol)
- X-θ hidden reversal (reverse flow only; θ protocol unavailable)

## Signature

Define signal S(N) = F_perfect(N) − F_hidden(N)

Predictions:

1. S(N) grows ~ linearly with |N| (topological lever).
2. S(N) is approximately odd: S(-N) ≈ -S(N) when θ̇ changes sign with winding direction.
3. Controls:
   - κ=0 ⇒ S(N) ≈ 0
   - N=0 ⇒ S(0) ≈ 0
4. Robustness:
   - signal persists across multiple dataset windows (error bars)

## Lab Translation (Quantum/Coherent Platform)

Implement θ as a controllable phase on a synthetic dimension (photonic lattice / cold atoms / spin systems).

- Prepare an initial wavepacket (analog of milk blob).
- Evolve under engineered gauge-like coupling Aθ(x).
- Apply an echo (reverse base Hamiltonian), with two cases:
  - reverse θ drive (perfect)
  - do not reverse θ drive (hidden)
    Measure return probability (Loschmidt echo) vs N.
